###### Content under Creative Commons Attribution license CC-BY 4.0, code under MIT license © 2015 L.A. Barba, G.F. Forsyth, B. Knaepen

# Relax and hold steady

This is the fourth and last notebook of **Module 5** (*"Relax and hold steady"*), dedicated to elliptic PDEs. In the [previous notebook](./05_03_Iterate.This.ipynb), we examined how different algebraic formulations can speed up the iterative solution of the Laplace equation, compared to the simplest (but slowest) Jacobi method.  The Gauss-Seidel and successive-over relaxation methods both provide faster algebraic convergence than Jacobi. But there is still room for improvement.  

In this lesson, we'll take a look at the very popular [conjugate gradient](https://en.wikipedia.org/wiki/Conjugate_gradient_method) (CG) method.  
The CG method solves linear systems with coefficients matrices that are symmetric and positive-definite. It is either used on its own, or in conjunction with multigrid—a technique that we'll explore later on its own (optional) course module.

For a real understanding of the CG method, there is no better option than studying the now-classic monograph by Jonathan Shewchuck: *"An introduction to the conjugate gradient method withouth the agonizing pain"* (1994). Here, we try to give you a brief summary to explain the implementation in Python.

## Head in the right direction!

### Test problem

Let's return to the Poisson equation example from [Lesson 2](./05_02_2D.Poisson.Equation.ipynb).

\begin{equation}
\nabla^2 p = -2\left(\frac{\pi}{2}\right)^2\sin\left( \frac{\pi x}{L} \right) \cos\left(\frac{\pi y}{L}\right)
\end{equation}

in the domain 

$$\left\lbrace \begin{align*}
0 &\leq x\leq 1  \\
-0.5 &\leq y \leq 0.5 
\end{align*} \right.$$

with boundary conditions 

$$p=0 \text{ at } \left\lbrace 
\begin{align*}
x&=0\\
y&=0\\
y&=-0.5\\
y&=0.5
\end{align*} \right.$$

We will solve this equation by assuming an initial state of $p=0$ everywhere, and applying boundary conditions to relax in pseudo-time.

### A bit of theory

The CG method aims to solve elliptic PDEs by iterating the unknown and having it converge, up to a given accuracy, to the solution dictated by the boundary conditions (it is a member of the so-called Krylov subspace methods, but this should not scare you...).

Recall that in its discretized form, the Poisson equation reads,

$$\frac{p_{i+1,j}^{k}-2p_{i,j}^{k}+p_{i-1,j}^{k}}{\Delta x^2}+\frac{p_{i,j+1}^{k}-2 p_{i,j}^{k}+p_{i,j-1}^{k}}{\Delta y^2}=b_{i,j}^{k}$$

You have manipulated equations of this kind in the past. Indeed, the left hand side represents a linear combination of the values of $p$ at several grid points and this linear combination has to be equal to the value of the source term, $b$, on the right hand side.

Now imagine you gather the values $p_{i,j}$ of $p$ at all grid points into a big vector ${\bf p}$ and you do the same for $b$ using the same ordering. Both vectors ${\bf p}$ and ${\bf b}$ contain $N=nx*ny$ values and thus belong to $\mathbf{R}^N$. The linear relationship we just described means that the discretised Poisson equation can be symbolically written as

\begin{equation}
A{\bf p}={\bf b}
\end{equation}

where $A$ is a $N\times N$ matrix. This is nothing other than a linear system of equations in matrix form. Although we will not directly use this form in the actual CG algorithm below, it is useful to examine the problem this way to understand how the method works.

The starting point is to define the iteration of the potential $p$ as

\begin{equation}
{\bf p}^{k+1}={\bf p}^k + \alpha {\bf d}^k
\end{equation}

At iteration $n+1$, ${\bf p}^k$ is modified by taking a step in the direction of ${\bf d}^k$. The idea is to start with a guess ${\bf p}^0$ and to march towards the solution by making jumps along the direction vectors ${\bf d}^k$.  $\alpha$ is a scalar that determines how big a jump to take at each iteration.  Now we need to carefully choose the direction vectors and the size of the jumps.

## The residual

One of the tools we use to determine the direction vectors is called the residual.  What is the residual?  We're glad you asked!

If we assume that a solution exists to the Poisson equation

\begin{equation}
A \bf p = b
\end{equation}

then we know that at each point in the domain, there will be some error between the calculated value, $p^k_i$ and the exact solution $p^{exact}_i$.  We may not know what the exact solution is, but we know it's out there.  The error at any point $i$ is the difference between the calculated value and the exact solution:

\begin{equation}
e^k_i = p^k_i - p^{exact}_i
\end{equation}

**Note:** We are talking about error at a specific point, not a measure of error across the entire domain.  

What if we recast the Poisson equation in terms of a not-perfectly-relaxed $\bf p^k$?

\begin{equation}
A \bf p^k \approx b
\end{equation}

We write this as an approximation because $\bf p^k \neq p$.  To "fix" the equation, we need to add an extra term to account for the difference in the Poisson equation $-$ that extra term is called the residual.  We can write out the modified Poisson equation like this:

\begin{equation}
{\bf r^k} + A \bf p^k = b
\end{equation}

## The method of steepest descent

Before considering the more complex CG algorithm, it is helpful to introduce a simpler approach called the method of steepest descent. At iteration $0$, we choose an inital guess. Unless we are immensely lucky, it will not satisfy the Poisson equation and we will have,

\begin{equation}
{\bf b}-A{\bf p}^0={\bf r}^0\ne {\bf 0}
\end{equation}

The vector ${\bf r}^0$ is called the initial residual and measures how far we are from satisfying the linear system. In fact, we can define such a residual vector at each iteration,

\begin{equation}
{\bf r}^k={\bf b}-A{\bf p}^k
\end{equation}

and monitor how this vector tends to zero as we iterate.

In the method of steepest descent, two choices are made:

1. the direction vectors are chosen to be the residuals ${\bf d}^k = {\bf r}^k$.
2. the length of the jumps are such that the $n+1^{th}$ residual is orthogonal to the $n^{th}$ resuidual.

<br>
<div class="alert alert-success">
**Note:** A consequence of choice 1 is that the direction of jump $n+1$ is orthogonal to the direction of jump $n$.
</div>

There are good (and, in fact, not complex) reasons to justify these choices and you should read one of the references supplied if you want to understand them. But since we want you to converge to the end of the notebook in a finite time, please accept them for now. 

Condition 2 requires that,

\begin{align}
{\bf r}^{k+1}\cdot {\bf r}^{k} = 0 \nonumber \\
\Leftrightarrow ({\bf b}-A{\bf p}^{k+1}) \cdot {\bf r}^{k} = 0 \nonumber \\
\Leftrightarrow ({\bf b}-A({\bf p}^{k}+\alpha {\bf r}^k)) \cdot {\bf r}^{k} = 0 \nonumber \\
\Leftrightarrow ({\bf r}^k+\alpha A{\bf r}^k) \cdot {\bf r}^{k} = 0 \nonumber \\
\alpha = \frac{{\bf r}^k \cdot {\bf r}^k}{A{\bf r}^k \cdot {\bf r}^k}.
\end{align}

We are now ready to test this algorithm.

To begin, let's import libraries, setup our mesh and import some helper functions:

In [ ]:
import numpy
from math import pi
%matplotlib inline

In [ ]:
from laplace_helper import plot_3D, L2_rel_error
from cg_helper import poisson_2d, p_analytical

In [ ]:
# Parameters
nx = 101
ny = 101
xmin = 0
xmax = 1
ymin = -0.5
ymax = 0.5

l2_target = 1e-10

# Spacing
dx = (xmax-xmin)/(nx-1)
dy = (ymax-ymin)/(ny-1)

# Mesh
x  = numpy.linspace(xmin,xmax,nx)
y  = numpy.linspace(ymin,ymax,ny)
X,Y = numpy.meshgrid(x,y)

# Source
L = xmax-xmin
b = -2*(pi/L)**2*numpy.sin(pi*X/L)*numpy.cos(pi*Y/L)

# Initialization
p_i  = numpy.zeros((ny,nx))

# Analytical solution
pan = p_analytical(X,Y,L)


### Time to implement steepest descent!  

Let's quickly review the solution process:

1. Calculate the residual, $\bf r^k$, which also serves as the direction vector, $\bf d^k$
2. Calculate the step size $\alpha$
3. Update ${\bf p}^{k+1}={\bf p}^k + \alpha {\bf d}^k$


##### How do we calculate the residual?  

We have an equation for the residual above: 
\begin{equation}
{\bf r}^k={\bf b}-A{\bf p}^k
\end{equation}

Remember that $A$ is just a stand-in for the Laplacian $\nabla^2$ and since $dx=dy$ we know that the Laplacian can be discretized as

\begin{equation}
\nabla^2 p^k = 4p^k_{i,j} - \left(p^{k}_{i,j-1} + p^k_{i,j+1} + p^{k}_{i-1,j} + p^k_{i+1,j} \right)
\end{equation}

##### What about calculating $\alpha$?

The calculation of $\alpha$ is relatively straightforward, but does require evaluating the term $A{\bf r^k}$.  That said, we just went over how to handle discretizing the $A$ operator.

Given that, you shouldn't have too much trouble writing out the discretization for $A{\bf r^k}$

In [ ]:
def steepest_descent_2d(p, b, dx, dy, l2_target):
    '''
    Performs steepest descent relaxation
    Assumes Dirichlet boundary conditions p=0
    
    Parameters:
    ----------
    p : 2D array of floats
        Initial guess
    b : 2D array of floats
        Source term
    dx: float
        Mesh spacing in x direction
    dy: float
        Mesh spacing in y direction
    l2_target: float
        Error target
    
    Returns:
    -------
    p: 2D array of float
        Distribution after relaxation
    '''
    r  = numpy.zeros((ny,nx)) # residual
    Ar  = numpy.zeros((ny,nx)) # to store result of matrix multiplication
    
    l2_norm = 1
    iterations = 0
    l2_conv = []
    
    # Iterations
    while l2_norm > l2_target:

        pd = p.copy()
        
        r[1:-1,1:-1] = b[1:-1,1:-1]*dx**2 + 4*pd[1:-1,1:-1] - \
            pd[1:-1,2:] - pd[1:-1,:-2] - pd[2:, 1:-1] - pd[:-2, 1:-1]
        
        Ar[1:-1,1:-1] = -4*r[1:-1,1:-1]+r[1:-1,2:]+r[1:-1,:-2]+\
            r[2:, 1:-1] + r[:-2, 1:-1]

        rho = numpy.sum(r*r)
        sigma = numpy.sum(r*Ar)
        alpha = rho/sigma

        p = pd + alpha*r
        
        # BCs are automatically enforced
        
        l2_norm = L2_rel_error(pd,p)
        iterations += 1
        l2_conv.append(l2_norm)
    
    print('Number of SD iterations: {0:d}'.format(iterations))
    return p, l2_conv     



Let's see how it performs on our example problem.

In [ ]:
p, l2_conv = steepest_descent_2d(p_i.copy(), b, dx, dy, l2_target)
L2_rel_error(p, pan)

Not bad! it took only two iterations to reach a solution that meets our steady state check. Although this seems great, the steepest descent algorithm is not great when used with large systems or more complex right hand sides in the Poisson equation (we'll examine this below!). We can get better performance if we take a little more care in selecting the direction vectors, $\bf d^k$.

## The method of conjugate gradients

With steepest descent, we know that two **successive** jumps are orthogonal, but that's about it.  There is nothing to prevent the algorithm from making several jumps in the same (or a similar) direction.  Imagine you wanted to go from the intersection of 5th Avenue and 23rd Street to the intersection of 9th Avenue and 30th Street. Knowing that each segment has the same computational cost (one iteration), would you follow the red path or the green path?

<img src="./figures/jumps.png" width=350>

#### Figure 1. Do you take the red path or the green path?

The method of conjugate gradients reduces the number of jumps by making sure the algorithm never selects the same direction twice. The size of the jumps is now given by:

\begin{equation}
\alpha = \frac{{\bf r}^k \cdot {\bf r}^k}{A{\bf d}^k \cdot {\bf d}^k}
\end{equation}

and the direction vectors by:

\begin{equation}
{\bf d}^{k+1}={\bf r}^{k+1}+\beta{\bf d}^{k}
\end{equation}

where $\beta = \frac{{\bf r}^{k+1} \cdot {\bf r}^{k+1}}{{\bf r}^k \cdot {\bf r}^k}$.

The search directions are no longer equal to the residuals but are instead a  linear combination of the residual and the previous search direction. It can be shown that this algorithm necessarily converges to the exact solution (up to machine accuracy) in a maximum of $N$ iterations. When one is satisfied with an approximate solution, the number of steps is significantly lower. Again, the derivation of the algorithm is not immensely difficult and can be found in Shewchuk (1994).

Here is a function implementing the CG algorithm:

### Implementing Conjugate Gradients

We will again update $\bf p$ according to 

\begin{equation}
{\bf p}^{k+1}={\bf p}^k + \alpha {\bf d}^k
\end{equation}

but use the modified equations above to calculate $\alpha$ and ${\bf d}^k$.  

You may have noticed that $\beta$ depends on both ${\bf r}^{k+1}$ and ${\bf r}^k$ and that makes the calculation of ${\bf d}^0$ a little bit tricky.  Or impossible (using the formula above).  Instead we set ${\bf d}^0 = {\bf r}^0$ for the first step and then switch for all subsequent iterations.  

Thus, the full set of steps for the method of conjugate gradients is:

Calculate ${\bf d}^0 = {\bf r}^0$ (just the once), then

1. Calculate $\alpha = \frac{{\bf r}^k \cdot {\bf r}^k}{A{\bf d}^k \cdot {\bf d}^k}$
2. Update ${\bf p}^{k+1}$
3. Calculate ${\bf r}^{k+1} = {\bf r}^k - \alpha A {\bf d}^k$ $\ \ \ \ $(see <a href='#references'>Shewchuk (1994)</a>)
4. Calculate $\beta = \frac{{\bf r}^{k+1} \cdot {\bf r}^{k+1}}{{\bf r}^k \cdot {\bf r}^k}$
5. Calculate ${\bf d}^{k+1}={\bf r}^{k+1}+\beta{\bf d}^{k}$
6. Repeat!

In [ ]:
def conjugate_gradient_2d(p, b, dx, dy, l2_target):
    '''Performs cg relaxation
    Assumes Dirichlet boundary conditions p=0
    
    Parameters:
    ----------
    p : 2D array of floats
        Initial guess
    b : 2D array of floats
        Source term
    dx: float
        Mesh spacing in x direction
    dy: float
        Mesh spacing in y direction
    l2_target: float
        Error target
    
    Returns:
    -------
    p: 2D array of float
        Distribution after relaxation
    '''
    r  = numpy.zeros((ny,nx)) # residual
    Ad  = numpy.zeros((ny,nx)) # to store result of matrix multiplication 
    
    l2_norm = 1
    iterations = 0
    l2_conv = []
    
    # Step-0 We compute the initial residual and 
    # the first search direction is just this residual
    
    r[1:-1,1:-1] = b[1:-1,1:-1]*dx**2 + 4*p[1:-1,1:-1] - \
        p[1:-1,2:] - p[1:-1,:-2] - p[2:, 1:-1] - p[:-2, 1:-1]
    d = r.copy()
    rho = numpy.sum(r*r)
    Ad[1:-1,1:-1] = -4*d[1:-1,1:-1]+d[1:-1,2:]+d[1:-1,:-2]+\
        d[2:, 1:-1] + d[:-2, 1:-1]
    sigma = numpy.sum(d*Ad)
    
    # Iterations
    while l2_norm > l2_target:

        pk = p.copy()
        rk = r.copy()
        dk = d.copy()
        
        alpha = rho/sigma

        p = pk + alpha*dk
        r = rk- alpha*Ad
        
        rhop1 = numpy.sum(r*r)
        beta = rhop1 / rho
        rho = rhop1
        
        d = r + beta*dk
        Ad[1:-1,1:-1] = -4*d[1:-1,1:-1] + d[1:-1,2:] + d[1:-1,:-2] + \
            d[2:, 1:-1] + d[:-2, 1:-1]
        sigma = numpy.sum(d*Ad)
        
        # BCs are automatically enforced
        
        l2_norm = L2_rel_error(pk,p)
        iterations += 1
        l2_conv.append(l2_norm)
    
    print('Number of CG iterations: {0:d}'.format(iterations))
    return p, l2_conv     


In [ ]:
p, l2_conv = conjugate_gradient_2d(p_i.copy(), b, dx, dy, l2_target)
L2_rel_error(p,pan)

The method of conjugate gradients also took two iterations to reach a solution that meets our steady state check.

Let's compare this to the number of iterations needed for the Jacobi iteration:

In [ ]:
p, l2_conv = poisson_2d(p_i.copy(), b, dx, dy, l2_target)

For the problem we considered, we observed a significant gain in terms of computational cost using the method of steepest descent or the conjugate gradient method.

## Complex Poisson problems

The conjugate gradient method really shines when one needs to solve more complex Poisson problems. To get an insight into this, let's solve the Poisson problem using the same boundary conditions as the previous problem but with the following right hand side,

\begin{equation}
b = \sin\left(\frac{\pi x}{L}\right) \cos\left(\frac{\pi y}{L}\right) + \sin\left(\frac{6\pi x}{L}\right) \cos\left(\frac{6\pi y}{L}\right)
\end{equation}


In [ ]:
b = (numpy.sin(pi*X/L)*numpy.cos(pi*Y/L)+
     numpy.sin(6*pi*X/L)*numpy.sin(6*pi*Y/L))

In [ ]:
p, l2_conv = poisson_2d(p_i.copy(), b, dx, dy, l2_target)

p, l2_conv = steepest_descent_2d(p_i.copy(), b, dx, dy, l2_target)

p, l2_conv = conjugate_gradient_2d(p_i.copy(), b, dx, dy, l2_target)

## References

<a id='references'></a>
Shewchuk, J. (1994). [An Introduction to the Conjugate Gradient Method Without the Agonizing Pain (PDF)](http://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf)

Ilya Kuzovkin, [The Concept of Conjugate Gradient Descent in Python](http://ikuz.eu/2015/04/15/the-concept-of-conjugate-gradient-descent-in-python/)

In [ ]:
from IPython.core.display import HTML
css_file = '../../styles/numericalmoocstyle.css'
HTML(open(css_file, "r").read())